In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import csv
import matplotlib.pyplot as plt
import tensorflow as tf

In [ ]:
timestep=[]
temp=[]
step=0
with open('/kaggle/input/sunspots/Sunspots.csv') as csvfile:
    reader=csv.reader(csvfile,delimiter=',')
    next(reader)
    
    for row in reader:
        temp.append(float(row[2]))
        timestep.append(step)
        step+=1
    
    

In [ ]:
plt.figure(figsize=(12,6))
plt.plot(timestep,temp)

In [ ]:
series=np.array(temp)
time=np.array(timestep)

split_time=3000
time_train=time[:split_time]
temp_train=series[:split_time]
time_valid=time[split_time:]
temp_valid=series[split_time:]

In [ ]:
window_size = 30
batch_size = 32
shuffle_buffer_size = 1000

In [ ]:
def windowed_set(series,batch_size,window_size,shuffle_buffer_size):
    series=tf.expand_dims(series,axis=-1)
    df=tf.data.Dataset.from_tensor_slices(series)
    df=df.window(window_size+1,shift=1,drop_remainder=True)
    df=df.flat_map(lambda window:window.batch(window_size+1))
    df=df.shuffle(shuffle_buffer_size).map(lambda window:(window[:-1],window[-1]))
    df=df.batch(batch_size).prefetch(1)
    return df

In [ ]:
train_series=windowed_set(temp_train,100,60,1000)

In [ ]:
train_series

In [ ]:
model=tf.keras.Sequential()
model.add(tf.keras.layers.Conv1D(filters=32,kernel_size=5,padding='causal',activation='relu',strides=1,input_shape=[None,1]))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,return_sequences=True)))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)))
model.add(tf.keras.layers.Dense(30,activation='relu'))
model.add(tf.keras.layers.Dense(10,activation='relu'))
model.add(tf.keras.layers.Dense(1))
model.add(tf.keras.layers.Lambda(lambda x:x *400))


In [ ]:
model.summary()

In [ ]:
lr_scheduler=tf.keras.callbacks.LearningRateScheduler(lambda epoch:1e-8*10**(epoch/20))

In [ ]:
optimizer = tf.keras.optimizers.SGD(lr=1e-6, momentum=0.9)

model.compile(loss=tf.keras.losses.Huber(),optimizer=optimizer,metrics=['mae'])

In [ ]:
history=model.fit(train_series,epochs=500)

In [ ]:
plt.semilogx(history.history['lr'],history.history['loss'])

In [ ]:
forecast=[]
for time in range(len(series)-60):
    serie1=series[time:time+60][np.newaxis]
    serie1=tf.expand_dims(serie1,axis=-1)
    forecast.append(model.predict(serie1))


In [ ]:
results=np.array(forecast)[:,0,0]

In [ ]:
len(results[split_time-60:])


In [ ]:
plt.plot(time_valid,temp_valid)
plt.plot(time_valid,results[split_time-60:])

In [ ]:
tf.keras.metrics.mean_absolute_error(temp_valid,results[split_time-60:]).numpy()
